In [11]:
import pandas as pd
import numpy as np
import shutil
import os

In [12]:
y_train = pd.read_csv('train_labels.csv')
y_test = pd.read_csv('test_labels.csv')

In [13]:
y_train

,class,filename,ymax,xmax,ymin,xmin,width,height
0,Bernese_mountain_dog,n02107683_5663.jpg,256,399,1,89,500,375
1,Scotch_terrier,n02097298_9432.jpg,374,439,12,11,500,375
2,Yorkshire_terrier,n02094433_1772.jpg,413,373,6,13,385,415
3,Yorkshire_terrier,n02094433_923.jpg,363,422,32,36,500,500
4,Affenpinscher,n02110627_6414.jpg,193,205,28,8,250,197
...,...,...,...,...,...,...,...,...
16459,Standard_schnauzer,n02097209_2813.jpg,333,392,38,24,500,334
16460,Norwegian_elkhound,n02091467_4636.jpg,398,291,0,10,300,405
16461,Samoyed,n02111889_5452.jpg,373,282,150,0,500,375
16462,Yorkshire_terrier,n02094433_2901.jpg,240,365,78,220,500,375


In [14]:
y_train

,class,filename,ymax,xmax,ymin,xmin,width,height
0,Standard_poodle,n02113799_3978.jpg,427,429,50,69,500,443
1,Basset,n02088238_10183.jpg,364,375,67,0,500,375
2,Whippet,n02091134_17567.jpg,312,304,82,49,500,358
3,Kelpie,n02105412_6154.jpg,404,330,18,30,350,426
4,Keeshond,n02112350_7115.jpg,289,340,179,114,480,424
...,...,...,...,...,...,...,...,...
4111,Siberian_husky,n02110185_14597.jpg,383,233,6,21,375,500
4112,Bouvier_des_Flandres,n02106382_1925.jpg,173,154,0,0,174,174
4113,Japanese_spaniel,n02085782_2428.jpg,798,586,22,18,600,800
4114,Dandie_Dinmont,n02096437_926.jpg,288,453,102,20,500,375


In [15]:
def read_class_map(path):
    cl_map = pd.read_csv(path,index_col=0).to_dict()['label']
    return cl_map

In [16]:
def from_xyxy_to_yolo(df):
    class_ids =  sorted(set(y_train['class']))
    pd.DataFrame({'label':class_ids}).to_csv('class_mapping.csv')
    class_mapping = dict(enumerate(class_ids))
    class_mapping_rev = {v: k for k,v in class_mapping.items()}
    df['label'] = df['class'].apply(lambda x: class_mapping_rev[x])
    df['x_center'] = df.apply(lambda x: ((x.xmax + x.xmin)/2)/x.width, axis=1)
    df['y_center'] = df.apply(lambda x: ((x.ymax + x.ymin)/2)/x.height, axis=1)
    df['bb_height'] = df.apply(lambda x: (x.ymax - x.ymin)/x.height, axis=1)
    df['bb_width'] = df.apply(lambda x: (x.xmax - x.xmin)/x.width, axis=1)
    return df

In [17]:
df_train = from_xyxy_to_yolo(y_train)
df_train

,class,filename,ymax,xmax,ymin,xmin,width,height,label,x_center,y_center,bb_height,bb_width
0,Bernese_mountain_dog,n02107683_5663.jpg,256,399,1,89,500,375,11,0.488000,0.342667,0.680000,0.620000
1,Scotch_terrier,n02097298_9432.jpg,374,439,12,11,500,375,96,0.450000,0.514667,0.965333,0.856000
2,Yorkshire_terrier,n02094433_1772.jpg,413,373,6,13,385,415,119,0.501299,0.504819,0.980723,0.935065
3,Yorkshire_terrier,n02094433_923.jpg,363,422,32,36,500,500,119,0.458000,0.395000,0.662000,0.772000
4,Affenpinscher,n02110627_6414.jpg,193,205,28,8,250,197,0,0.426000,0.560914,0.837563,0.788000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16459,Standard_schnauzer,n02097209_2813.jpg,333,392,38,24,500,334,106,0.416000,0.555389,0.883234,0.736000
16460,Norwegian_elkhound,n02091467_4636.jpg,398,291,0,10,300,405,80,0.501667,0.491358,0.982716,0.936667
16461,Samoyed,n02111889_5452.jpg,373,282,150,0,500,375,94,0.282000,0.697333,0.594667,0.564000
16462,Yorkshire_terrier,n02094433_2901.jpg,240,365,78,220,500,375,119,0.585000,0.424000,0.432000,0.290000


In [18]:
df_test = from_xyxy_to_yolo(y_test)
df_test

,class,filename,ymax,xmax,ymin,xmin,width,height,label,x_center,y_center,bb_height,bb_width
0,Standard_poodle,n02113799_3978.jpg,427,429,50,69,500,443,105,0.498000,0.538375,0.851016,0.720000
1,Basset,n02088238_10183.jpg,364,375,67,0,500,375,8,0.375000,0.574667,0.792000,0.750000
2,Whippet,n02091134_17567.jpg,312,304,82,49,500,358,117,0.353000,0.550279,0.642458,0.510000
3,Kelpie,n02105412_6154.jpg,404,330,18,30,350,426,63,0.514286,0.495305,0.906103,0.857143
4,Keeshond,n02112350_7115.jpg,289,340,179,114,480,424,62,0.472917,0.551887,0.259434,0.470833
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4111,Siberian_husky,n02110185_14597.jpg,383,233,6,21,375,500,101,0.338667,0.389000,0.754000,0.565333
4112,Bouvier_des_Flandres,n02106382_1925.jpg,173,154,0,0,174,174,20,0.442529,0.497126,0.994253,0.885057
4113,Japanese_spaniel,n02085782_2428.jpg,798,586,22,18,600,800,61,0.503333,0.512500,0.970000,0.946667
4114,Dandie_Dinmont,n02096437_926.jpg,288,453,102,20,500,375,35,0.473000,0.520000,0.496000,0.866000


In [19]:
def create_yolo_ds(df,images_path, train = True):
    if train:
        ds_path = os.path.join(os.getcwd(),'train','')
    else:
        ds_path = os.path.join(os.getcwd(),'valid','')
    for i in range(len(df)):
        shutil.copyfile(os.path.join(images_path,df['filename'][i]), os.path.join(ds_path,"images",df['filename'][i]))
        yolo_annotation = f"{df['label'][i]} {df['x_center'][i]} {df['y_center'][i]} {df['bb_width'][i]} {df['bb_height'][i]}"
        with open(os.path.join(ds_path,'labels',df['filename'][i].split(".")[0]+".txt"),"w",encoding="utf8") as f:
            f.write(yolo_annotation+'\n')

In [20]:
create_yolo_ds(df_train,'images')

In [21]:
create_yolo_ds(df_test,'images', train = False)

In [22]:
cl_mapping = read_class_map('class_mapping.csv')
nc =  len(cl_mapping)
names =  list(cl_mapping.values())

with open("data.yaml", "w") as f:
    f.write(f"train: ../train/images\nval: ../valid/images\n\nnc: {nc}\nnames: {names}")